In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import optuna

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    make_scorer
)


In [ ]:
df_train = pd.read_csv("numeric_train_data.csv")
df_train.head(5)

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,label
0,0.617565,0.598418,0.607773,0.642276,0.652537,0.700823,0.668325,0.565349,0.578494,0.565391,...,0.737623,0.674724,0.675981,0.584386,0.655460,0.523524,0.527622,0.357415,0.552595,1
1,0.544064,0.542970,0.571765,0.614927,0.607819,0.487641,0.455840,0.495413,0.590330,0.528084,...,0.721271,0.520178,0.600715,0.540846,0.458964,0.747960,0.383462,0.591288,0.521658,1
2,0.512500,0.549178,0.594594,0.559915,0.483045,0.524708,0.590397,0.638933,0.801045,0.738217,...,0.625295,0.544563,0.495568,0.566445,0.638160,0.529887,0.542644,0.484434,0.384409,1
3,0.493417,0.405156,0.208711,0.285280,0.365346,0.364058,0.296190,0.263013,0.286027,0.387489,...,0.520085,0.420716,0.331187,0.559944,0.427646,0.627307,0.331686,0.451716,0.559956,1
4,0.307183,0.225640,0.253133,0.300230,0.418234,0.524504,0.560558,0.696728,0.676684,0.476201,...,0.515845,0.618530,0.419984,0.623399,0.537840,0.463290,0.470429,0.444843,0.354809,0


In [3]:
df_train.groupby(by="label").count()

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20
label,,,,,,,,,,,,,,,,,,,,,
0,9986,9986,9986,9986,9986,9986,9986,9986,9986,9986,...,9986,9986,9986,9986,9986,9986,9986,9986,9986,9986
1,12482,12482,12482,12482,12482,12482,12482,12482,12482,12482,...,12482,12482,12482,12482,12482,12482,12482,12482,12482,12482


In [ ]:
df_train = df_train
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]



In [6]:
X_train.head()

,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20
0,0.617565,0.598418,0.607773,0.642276,0.652537,0.700823,0.668325,0.565349,0.578494,0.565391,...,0.432913,0.737623,0.674724,0.675981,0.584386,0.655460,0.523524,0.527622,0.357415,0.552595
1,0.544064,0.542970,0.571765,0.614927,0.607819,0.487641,0.455840,0.495413,0.590330,0.528084,...,0.325626,0.721271,0.520178,0.600715,0.540846,0.458964,0.747960,0.383462,0.591288,0.521658
2,0.512500,0.549178,0.594594,0.559915,0.483045,0.524708,0.590397,0.638933,0.801045,0.738217,...,0.523860,0.625295,0.544563,0.495568,0.566445,0.638160,0.529887,0.542644,0.484434,0.384409
3,0.493417,0.405156,0.208711,0.285280,0.365346,0.364058,0.296190,0.263013,0.286027,0.387489,...,0.370275,0.520085,0.420716,0.331187,0.559944,0.427646,0.627307,0.331686,0.451716,0.559956
4,0.307183,0.225640,0.253133,0.300230,0.418234,0.524504,0.560558,0.696728,0.676684,0.476201,...,0.583253,0.515845,0.618530,0.419984,0.623399,0.537840,0.463290,0.470429,0.444843,0.354809


### RANDOM FOREST

In [ ]:
feature_rf = [
    "chroma_2", "chroma_3", "chroma_4", "chroma_6", "chroma_7",
    "chroma_8", "chroma_9", "chroma_10", "chroma_11", "chroma_12",
    "rms_mean", "spectral_centroid_mean", "spectral_bandwidth_mean",
    "spectral_rolloff_mean", "zcr_mean",
    "mfcc_1", "mfcc_2", "mfcc_3", "mfcc_4", "mfcc_5", "mfcc_6",
    "mfcc_7", "mfcc_8", "mfcc_9", "mfcc_10", "mfcc_11", "mfcc_12",
    "mfcc_13", "mfcc_14", "mfcc_15", "mfcc_16", "mfcc_17", "mfcc_18",
    "mfcc_19", "mfcc_20"
]

X_train_rf = X_train[feature_rf]

Optuna

In [ ]:
# 1. Subset fitur untuk training
X_train_rf = X_train[feature_rf]

# 2. Objective function untuk tuning Random Forest
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5]),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'n_jobs': -1,
        'random_state': 42
    }

    model_rf = RandomForestClassifier(**params)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        model_rf,
        X_train_rf,  # ← gunakan subset fitur
        y_train,
        scoring=make_scorer(accuracy_score),
        cv=cv,
        n_jobs=-1
    )

    return scores.mean()



In [10]:
# 3. Jalankan Optimasi
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # bisa diubah sesuai kebutuhan

# 4. Tampilkan hasil terbaik
print("Best trial:")
print(study.best_trial)


[I 2025-05-28 03:04:19,856] A new study created in memory with name: no-name-6907877f-062c-4812-81b0-e39a78b88bde
[I 2025-05-28 03:04:39,382] Trial 0 finished with value: 0.9088482395252429 and parameters: {'n_estimators': 139, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'class_weight': None}. Best is trial 0 with value: 0.9088482395252429.
[I 2025-05-28 03:05:33,848] Trial 1 finished with value: 0.9116077712143035 and parameters: {'n_estimators': 136, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 0.5, 'class_weight': None}. Best is trial 1 with value: 0.9116077712143035.
[I 2025-05-28 03:06:03,857] Trial 2 finished with value: 0.8857039546558653 and parameters: {'n_estimators': 292, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'class_weight': None}. Best is trial 1 with value: 0.9116077712143035.
[I 2025-05-28 03:06:58,134] Trial 3 finished with value: 0.82748782633

Best trial:
FrozenTrial(number=48, state=1, values=[0.9163700424662962], datetime_start=datetime.datetime(2025, 5, 28, 4, 6, 55, 360886), datetime_complete=datetime.datetime(2025, 5, 28, 4, 8, 52, 735601), params={'n_estimators': 278, 'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.5, 'class_weight': None}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=100, step=1), 'max_depth': IntDistribution(high=20, log=False, low=5, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=5, log=False, low=1, step=1), 'max_features': CategoricalDistribution(choices=('sqrt', 'log2', 0.5)), 'class_weight': CategoricalDistribution(choices=(None, 'balanced'))}, trial_id=48, value=None)


In [ ]:
# Simpan model terbaik
joblib.dump(best_model, 'best_model_rf_newver.pkl')
print("✅ Model berhasil disimpan")

✅ Model berhasil disimpan


### SVM

In [ ]:
feature_svm = [
    "chroma_1", "chroma_2", "chroma_4", "chroma_9", "chroma_10", "chroma_11",
    "rms_mean", "spectral_centroid_mean", "spectral_bandwidth_mean",
    "spectral_rolloff_mean", "zcr_mean",
    "mfcc_1", "mfcc_2", "mfcc_3", "mfcc_4", "mfcc_5", "mfcc_6", "mfcc_7",
    "mfcc_9", "mfcc_11", "mfcc_12", "mfcc_13", "mfcc_14", "mfcc_15", "mfcc_18"
]

X_train_svm = X_train[feature_svm]

In [ ]:
# 1. Subset fitur untuk SVM
X_train_svm = X_train[feature_svm]

# 2. Objective function untuk tuning SVM
def objective(trial):
    params = {
        'C': trial.suggest_float('C', 1e-2, 1e2, log=True),
        'gamma': trial.suggest_float('gamma', 1e-4, 1e0, log=True),
        'kernel': 'rbf',
        'random_state': 42
    }

    model_svm = SVC(**params)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        model_svm,
        X_train_svm,  # ← gunakan fitur subset SVM
        y_train,
        scoring=make_scorer(accuracy_score),
        cv=cv,
        n_jobs=-1
    )

    return scores.mean()



In [17]:
# 3. Jalankan optimasi
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 4. Lihat hasil terbaik
print("Best trial:")
print(study.best_trial)


[I 2025-05-28 04:19:01,479] A new study created in memory with name: no-name-70d2094d-760b-4a8f-9e19-82f1e1ca6d6c
[I 2025-05-28 04:19:34,353] Trial 0 finished with value: 0.807192100533976 and parameters: {'C': 0.0153955033871323, 'gamma': 0.1678134785087825}. Best is trial 0 with value: 0.807192100533976.
[I 2025-05-28 04:19:57,455] Trial 1 finished with value: 0.8448458567453638 and parameters: {'C': 1.2686285828263062, 'gamma': 0.035971559479165124}. Best is trial 1 with value: 0.8448458567453638.
[I 2025-05-28 04:20:28,330] Trial 2 finished with value: 0.8139574085030257 and parameters: {'C': 0.029207005005552563, 'gamma': 0.12352248841716834}. Best is trial 1 with value: 0.8448458567453638.
[I 2025-05-28 04:21:11,183] Trial 3 finished with value: 0.58055890927003 and parameters: {'C': 0.622422510000506, 'gamma': 0.00037648066648786685}. Best is trial 1 with value: 0.8448458567453638.
[I 2025-05-28 04:21:27,449] Trial 4 finished with value: 0.8893983431280287 and parameters: {'C': 

Best trial:
FrozenTrial(number=30, state=1, values=[0.9383568525870881], datetime_start=datetime.datetime(2025, 5, 28, 4, 30, 31, 286575), datetime_complete=datetime.datetime(2025, 5, 28, 4, 30, 54, 729566), params={'C': 98.03363720445789, 'gamma': 0.9926573491055003}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=100.0, log=True, low=0.01, step=None), 'gamma': FloatDistribution(high=1.0, log=True, low=0.0001, step=None)}, trial_id=30, value=None)


In [ ]:
# Simpan model terbaik
joblib.dump(best_model_svm, 'best_model_svm_newver.pkl')
print("✅ Model berhasil disimpan")

✅ Model berhasil disimpan
